### King County - start urban farming club 

#### close to a city and affordable houses with large lots

In [ ]:
#imports
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
sns.set(style='white')

### background check on the region to pick the citys of intrest

In [ ]:
us_cities = pd.read_csv('data/uscities.csv')

In [ ]:
# load the citys from : https://simplemaps.com/data/us-cities
c = us_cities[(us_cities.state_id == 'WA') & (us_cities.county_name == 'King')]

# main filter 20k population - leave seattle out
cns = c[(c.city != 'Seattle') & (c.population > 20000)]

# Plot
plt.figure(figsize=(12,8))
plt.bar(cns.city,cns.population)
plt.xticks(cns.city, rotation='vertical', size=12)
plt.xlabel('city')
plt.ylabel('population')
plt.title(f'kink county city pop. (without Seattle {c.population.max()})')
plt.gcf().subplots_adjust(bottom=0.25)
plt.savefig('./pic_exports/CitySelect_Population_no_Seattle', dpi=300)
plt.show()
cns.shape

In [ ]:
# save the reduced data set
Selcted_cities = c[(c.population > 20000)]
Selcted_cities.shape
Selcted_cities.to_csv('data/King_county_city_data.csv')

### Load House price data and correlate to selected citys

In [ ]:
df_raw = pd.read_csv('data/King_County_House_prices_dataset.csv')
city_data =  pd.read_csv('data/King_county_city_data.csv')

In [ ]:
# Just check if the data some how match in lat and long...
plt.figure(figsize=(10,10))
plt.scatter(x='long',y='lat',data=df_raw[df_raw.long < -121.7])
plt.scatter(x='lng',y='lat',data=city_data)
plt.axis('equal')

#### Get distance to city 

In [ ]:
# add coordinats as tuple
df_raw['coord'] = list(zip(df_raw.lat, df_raw.long))
city_data['coord'] = list(zip(city_data.lat, city_data.lng))
df_raw.head()

In [ ]:
# this takes a little time !!!
# calc geo distance from city center to each house
import geopy.distance
def distto(c1, c2):
    return geopy.distance.vincenty(c1, c2).km

for cn in city_data['city']:
    city_coord = city_data[city_data.city == cn].coord
    df_raw[cn] = pd.Series(df_raw.coord.apply(lambda x: distto(x,city_coord)))

In [ ]:
# Verify coordinates
city = 'Seattle'
plt.figure(figsize=(10,10))
plt.scatter(x='long',y='lat',data=df_raw[df_raw.long < -121.7])
plt.scatter(x='long',y='lat',data=df_raw[df_raw[city] < 10])
plt.scatter(x='lng',y='lat',data=city_data[city_data.city == city])
d = city_data[city_data.city != city]
plt.scatter(x='lng',y='lat',data=d[d.city != city])
for c,lat,lng in zip(d.city,d.lat,d.lng):
    plt.text(lng,lat,c, fontsize=12)
plt.axis('equal')

### Unit column adding for EU units m^2

In [ ]:
Unit_sqf2sqm = 0.092903
Soccerfield = 68*105

In [ ]:
df_raw.columns

In [ ]:
df_raw['sqm_living'] =  df_raw.sqft_living * Unit_sqf2sqm
df_raw['sqm_lot'] =  df_raw.sqft_lot * Unit_sqf2sqm
df_raw['sqm_lot_soccer'] =  (df_raw.sqft_lot * Unit_sqf2sqm)/Soccerfield
df_raw['sqm_above'] =  df_raw.sqft_above * Unit_sqf2sqm
df_raw['sqm_living15'] =  df_raw.sqft_living15 * Unit_sqf2sqm
df_raw['sqm_lot15'] =  df_raw.sqft_lot15 * Unit_sqf2sqm

### Adding a price in million and thousend doller

In [ ]:
df_raw['Mprice'] = df_raw.price/1e6
df_raw['kprice'] = df_raw.price/1e3

## clean yr_renovated and add time since last renovated or age  

In [ ]:
# deal with NaN
df_raw.yr_renovated.isnull().value_counts()

In [ ]:
df_raw.yr_renovated[df_raw.yr_renovated.isnull()] = 0
df_raw.yr_renovated.isnull().value_counts()

In [ ]:
# add the removation_age 
# now - year renovated*is renovated - year build
def RenoAge(built,Reno):
    Now = 2015
    if Reno==0.:
        RenoAge = Now-built
    else:
        RenoAge = Now-Reno
    return RenoAge

df_raw['renovated'] = df_raw.yr_renovated!=0
df_raw['renovation_age'] = \
    df_raw.apply(lambda x: RenoAge(x.yr_built,x.yr_renovated), axis= 1)

df_raw[['renovation_age','renovated','yr_built','yr_renovated']].head()

In [ ]:
df_raw['renovation_age'].hist()

### Overall season of sales split date of sale 

In [ ]:
# get the month
import re
temp = df_raw['date']
temp = temp.apply(lambda x: int(re.findall('\d+',x)[0]))
df_raw['sale_month'] = temp

In [ ]:
# get the year
import re
temp = df_raw['date']
temp = temp.apply(lambda x: int(re.findall('\d+',x)[2]))
df_raw['sale_year'] = temp
df_raw.sale_year.unique() 
# sales dates are from 2014-2015

In [ ]:
# get impression over the price season ...
def plotMaxMeanMin(df_p,key,safeto=None):
    keys = [zipcode for zipcode, df in df_p.groupby(['sale_month'])]
    keys = [str(z) for z in keys]
    plt.bar(keys,df_p.groupby([key]).max()['Mprice'])
    plt.bar(keys,df_p.groupby([key]).mean()['Mprice'])
    plt.bar(keys,df_p.groupby([key]).min()['Mprice'])
    plt.ylabel('max/mean/min price in USD (M$)')
    plt.xlabel(key)
    plt.xticks(keys, rotation='vertical', size=10)
    if safeto:
        plt.savefig(safeto, dpi=300)
    plt.show()
    
plotMaxMeanMin(df_raw,'sale_month','./pic_exports/Sale_Season_Overall_1')

In [ ]:
plotMaxMeanMin(df_raw[df_raw.Mprice < 1],'sale_month','./pic_exports/Sale_Season_Overall_2')

### Selecting the rigion

- needed space at least 12000 m^2 for lot
- price limit somwehre ... 

In [ ]:
# price limit reasonabel 
g = sns.jointplot(x='sqm_lot',y='Mprice',data=df_raw[df_raw.price < 2.5e6],kind='scatter')
# good box is bellow

# draw the red line
x0, x1 = g.ax_joint.get_xlim()
y0, y1 = g.ax_joint.get_ylim()
price_limit = 1.25 # set price limit by eye
g.ax_joint.plot([x0,x1], [price_limit,price_limit], ':r')

plt.savefig('./pic_exports/ReasonablePrice', dpi=300)

here also a cut of for extream large lots would be a good idea for the possible linear rel. extraction...

In [ ]:
df_raw.sqm_lot.max()

### create basic stats for fitting house for all possible citys 
to genrate to find best city in regards to amount of found houses price limit and close to the city center
this also acts as outlyer filter indirectly...

In [ ]:
# TODO fill the rest of basic stats
lot_thres = 12000

df_stat = pd.DataFrame(columns=['hits','population','low_price','low_price_lot','mean_lot'], index=city_data.city)
for city in city_data.city:
    df = df_raw[(df_raw[city] < 10) & 
                (df_raw[city] > 2) & 
                (df_raw['sqm_lot'] > lot_thres) & 
                (df_raw['Mprice'] < price_limit)]

    df_stat.at[city,'hits'] = df.shape[0]
    df_stat.at[city,'low_price'] = df.price.min()
    df_stat.at[city,'low_price_lot'] = df[df.price == df.price.min()].sqft_lot.max()
    #print(city)
    a = city_data[city_data.city.str.contains(city)].population.max()
    df_stat.at[city,'population'] = a
    
df_stat.hits.plot.bar()

### find metric to mage city choice based on availability and proximity 

In [ ]:
fig, ax1 = plt.subplots(figsize=(12,8))
ax2 = ax1.twinx() # left and right y axis
keys = df_stat.index
ax2.plot(keys,df_stat.population/1000,'r')
ax1.bar(keys,df_stat.hits*df_stat.population/10000)
ax1.bar(keys,df_stat.hits*1)
ax1.set_xlabel('city')
ax2.set_ylabel('pop. / 1000', color='r')
ax2.set_ylim([0,200])
ax1.set_ylabel('hits and hits*pop./10000 ', color='b')
ax1.set_xticklabels(keys, rotation='vertical', size=12)

plt.savefig('./pic_exports/CitySelect_Pass2ByHitsWithMetric', dpi=300)
fig.show()

### continue with top 5 citys and create the dataframe with selected

In [ ]:
city=['Auburn','Kent', 'Sammamish', 'Maple Valley', 'Renton'] 
df_select = df_raw[(((df_raw[city[0]] < 10) & (df_raw[city[0]] > 1)) | 
             ((df_raw[city[1]] < 10) & (df_raw[city[1]] > 1)) |
             ((df_raw[city[4]] < 10) & (df_raw[city[4]] > 1)) |
             ((df_raw[city[2]] < 10) & (df_raw[city[2]] > 1)) |
             ((df_raw[city[3]] < 10) & (df_raw[city[3]] > 1)) ) & 
                (df_raw['sqm_lot'] > lot_thres) & 
                (df_raw['Mprice'] < price_limit)]

df_all_range = df_raw[(((df_raw[city[0]] < 10) & (df_raw[city[0]] > 1)) | 
             ((df_raw[city[1]] < 10) & (df_raw[city[1]] > 1)) |
             ((df_raw[city[4]] < 10) & (df_raw[city[4]] > 1)) |
             ((df_raw[city[2]] < 10) & (df_raw[city[2]] > 1)) |
             ((df_raw[city[3]] < 10) & (df_raw[city[3]] > 1)) )  ]

print(df_select.shape)
print(df_all_range.shape)

### Drawing on the Map to visulize the selection against all 

In [ ]:

## adding shape file to plot a card
import shapefile as shp  # Requires the pyshp package
sf = shp.Reader("./data/Zipcodes_for_King_County_and_Surrounding_Area__zipcode_area.shp")

#ploting the card as lines plot
plt.figure(figsize=(12,12))
for shape in sf.shapeRecords():
    x = [i[0] for i in shape.shape.points[:]]
    y = [i[1] for i in shape.shape.points[:]]
    plt.plot(x,y,'gray', alpha=0.6)
    
# ploting all, close range, and selction
plt.scatter(x='long',y='lat',data=df_raw[df_raw.long < -121.7],label= 'all', alpha=0.6)
plt.scatter(x='long',y='lat',data=df_all_range, label='all in range', alpha=0.5, color='g')
plt.scatter(x='long',y='lat',data=df_select, label='hits', color='r')

# adding city names
d = city_data
plt.scatter(x='lng',y='lat',data=d ,marker='+', label='city center', color='k')
for c,lat,lng in zip(d.city,d.lat,d.lng):
    plt.text(lng,lat,c, fontsize=15)
plt.legend()
#plt.axis('equal')

# finde tune the zoom
plt.ylim(47.1,47.1+0.8)
plt.xlim(-122.5,-122.5+0.8)

plt.savefig('./pic_exports/CitySelect_Map', dpi=300)

### check the price season

In [ ]:
plotMaxMeanMin(df_select[df_select.Mprice < 2],'sale_month','./pic_exports/CitySelectedSale_Season_Overall_2')

In [ ]:
plt.figure(figsize=(10,8))
sns.boxplot(x="sale_month", y="Mprice", data=df_select)
plt.savefig('./pic_exports/CitySelect_sales_month_box', dpi=300)

In [ ]:
# Check the freq. of sales per month

# TODO add better plot
df_select.sale_month.hist(bins=range(1,13))
plt.xticks(range(1,13))
plt.xlabel('month')
plt.ylabel('freq. sales')
plt.savefig('./pic_exports/CitySelect_sales_month_hist', dpi=300)

### calc the portion of the selection to indecate chances of getting a hit

In [ ]:
df_hit_chances = pd.DataFrame()
i = pd.Index(['all', 'selected', 'all_in_range'])
df_hit_chances['hits'] = pd.Series([df_raw.shape[0],df_select.shape[0],df_all_range.shape[0]],
                    index=i)

df_hit_chances['prop total'] = pd.Series([df_raw.shape[0]/df_raw.shape[0],df_select.shape[0]/df_raw.shape[0],df_all_range.shape[0]/df_raw.shape[0]],
                    index=i)

df_hit_chances['prop range'] = pd.Series([df_raw.shape[0]/df_all_range.shape[0],df_select.shape[0]/df_all_range.shape[0],df_all_range.shape[0]/df_all_range.shape[0]],
                    index=i)

#sns.heatmap(df_hit_chances)
df_hit_chances

In [ ]:
# TODO - calc chances make table
# add all in price range 
# all in size range
chances

### Looking for correlation - Find Price Indecators of houses

In [ ]:
df_select.columns

In [ ]:
l_focus = ['price',
           'sqm_lot',
           'renovation_age',
           'sqm_living',
           'yr_built',
           'Seattle',
           'Bellevue',
           'grade',
           'condition',
           'bathrooms',
           'sale_month',
           'floors']

plt.figure(figsize=(10,10))
sns.heatmap(df_select[l_focus].corr(),cmap='coolwarm',annot=True)

# all chosen seem to have a impact expet sale_month...

In [ ]:
sns.pairplot(df_select[l_focus])

In [ ]:
# try to scale the price log this has not been used in the end ...
df_select['priceLog'] = np.log(df_select.price)
l_focus[0] = 'priceLog'

In [ ]:
l_focus[0] = 'price'
l_focus.append('Kent') # patching cell

In [ ]:
# explore bi var to price
for e in l_focus:
    sns.lmplot(y=l_focus[0],x=e,data=df_select)
    plt.savefig('./pic_exports/Scatter_%s' %e, dpi=300)
    

In [ ]:
# Notebook cell to pick variables for the inital Model

# primary correlation
# sqm_living
# bathrooms
# grade ( can be kepts as is)
# renovation_age


# mini impact distance to the biggest citys
# just take Seattle


# special attention
# sqm_lot does not drive the price - TODO make plot for presentaion
# as expected sale month

# catecoricals 
#floors

#condtion
#(grade)

## build a OLS model - Tunning 

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import statsmodels.api as sm

#https://stats.stackexchange.com/questions/58391/mean-absolute-percentage-error-mape-in-scikit-learn
def mean_absolute_percentage_error(y_true, y_pred): 
    return np.round(np.mean(np.abs((y_true - y_pred) / y_true)) * 100,2)

In [ ]:
# define the variables
y = df_select['price']
#y = df_select['priceLog']
#X = df_select[['sqm_living','Seattle']]
#X = df_select[['sqm_living','bathrooms', 
#               'Seattle', 'grade', 'renovation_age' ]]

# Split Training set
X = df_select[['sqm_living','bathrooms', 'yr_built',
         'Seattle', 'grade', 'renovation_age' , 'Auburn','Kent', 'Sammamish', 'Maple Valley', 'Renton' ]]
X_train, X_test, y_train, y_test = \
train_test_split(X, y, test_size=0.3, random_state=101)

In [ ]:
# fit a model
lm = LinearRegression()
lm.fit(X_train,y_train)
#lm = sm.OLS(y_train ,X_train).fit() # sklearn has not the summery feedback ... can be toggled here
y_hat_t = lm.predict( X_train)

In [ ]:
# Do prediction and give some feedback on the fit
y_hat = lm.predict( X_test)
sns.regplot(y_test,y_hat)
plt.xlabel('Y Test')
plt.ylabel('Predicted Y')
plt.show()
sns.distplot((y_test-y_hat)/1000,bins=20);
plt.show()

In [ ]:
# explore coeffs of the model (sklearn model)
coeffecients = pd.DataFrame(lm.coef_,X.columns)
coeffecients.columns = ['Coeffecient']
coeffecients


In [ ]:
# try to make a simple indcator list to transport insite of the the coeff  (sklearn model)
# TODO better way would be to normilize before fitting...
def dir_exp(x):
    if x>0:
        return 'increase increases price'
    else:
        return 'increase reduces price'
coeffexp = coeffecients.Coeffecient.apply(lambda x: dir_exp(x))
coeffexp

In [ ]:
#get summery from statsmodel.api (not working with sklearn model !!) 
# lm.summary() 

In [ ]:
# build feedback Table on model qualty
ModelMetric = pd.DataFrame(columns=['Model Metric', 'Data Stat'])
ModelMetric.at['MAE($)','Model Metric'] = (metrics.mean_absolute_error(y_test, y_hat))
ModelMetric.at['MPE(%)','Model Metric'] = mean_absolute_percentage_error(y_test, (y_hat))
ModelMetric.at['MSE','Model Metric'] = metrics.mean_squared_error(y_test, y_hat)
ModelMetric.at['RMSE','Model Metric'] = np.sqrt(metrics.mean_squared_error(y_test, y_hat))
ModelMetric.at['R2','Model Metric'] = metrics.r2_score(y_test, y_hat)
ModelMetric.at['R2train','Model Metric'] = metrics.r2_score(y_train, y_hat_t)
ModelMetric.at['abs. MEAfMean','Model Metric'] = y_train.mean()*mean_absolute_percentage_error(y_test, y_hat)/100
ModelMetric.at['abs. MEAfMax','Model Metric'] = y_train.max()*mean_absolute_percentage_error(y_test, y_hat)/100
ModelMetric.at['abs. MEAfMean','Data Stat'] = (y_train.mean())
ModelMetric.at['abs. MEAfMax','Data Stat'] = (y_train.max())
ModelMetric

In [ ]:
# probing abslutes of the error...
print(np.exp(y_train.mean())*metrics.mean_absolute_error(y_test, y_hat))
print(np.exp(y_train.mean())*metrics.mean_absolute_error(y_train, y_hat_t))

## Try to Improve prediction

In [ ]:
## Test some feature combination to improve the fit

def Train(xlist,y,df):   
    y = df[y]
    X = df[xlist]
    X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.3, random_state=101)
    #lm = LinearRegression()
    lm = sm.OLS(y_train ,X_train).fit()
    #lm.fit(X_train,y_train)
    y_hat = lm.predict( X_test)
    y_hat_t = lm.predict( X_train)

    return mean_absolute_percentage_error(y_test, y_hat)

y = 'price'
xlist = ['sqm_living']
print(xlist ,Train(xlist,y,df_select))

y = 'price'
xlist = ['sqm_living', 'renovation_age']
print(xlist ,Train(xlist,y,df_select))


y = 'price'
xlist = ['sqm_living','bathrooms']
print(xlist ,Train(xlist,y,df_select))

y = 'price'
xlist = ['sqm_living','grade','Seattle']
print(xlist ,Train(xlist,y,df_select))


y = 'price'
xlist = ['grade','sqm_living','Seattle','renovation_age']
print(xlist ,Train(xlist,y,df_select))


y = 'price'
xlist = ['sqm_living','bathrooms', 
         'Seattle', 'grade', 'renovation_age'  ]
print(xlist ,Train(xlist,y,df_select))

y = 'price'
xlist = ['sqm_living','bathrooms', 
         'Seattle', 'grade', 'renovation_age' ,'Bellevue' ]
print(xlist ,Train(xlist,y,df_select))

y = 'price'
xlist = ['sqm_living','bathrooms', 
         'Seattle', 'grade', 'yr_built' , 'renovation_age' ,'Bellevue', 'Auburn','Kent', 'Sammamish', 'Maple Valley', 'Renton' ]
print(xlist ,Train(xlist,y,df_select))




# Reflection

## Model  and future work 
the model was only used on very small subset of the data a compare against the all county data etc. should be done.

the features used are not encoded for categorys like grade betroom etc at all - inital experments wehre explored but I returned to simpler means for now.

the pure distance to the city center is intresting but the zipcode category or binning into the city could lead to more correlation and predictive insite. 

There seems some more potential in cleaning the data i.e. very old buildings. Or the outlyers in lot size ...

The model is OLS linear for now no poly fit was tested. 

At one point experments with log of y was tested but on the reduced set the impact was not noticable (whiel experments in the Dev folder showed some promising results) 

To little work was done to clean the date from obvios outlyers and the 15 closed houses metrics wehre not used. Also the idea with soccerfield size compare for display was not used all because of time limits. 

The derived tendency of price increasing direction is partial based on to much variances and to little siginfacanse and need to redisigned as distance to a city is direction dependent and also clashes with other citys that are close ...


## Methode to find best city
even though the correlation of population and amount of found houses for the category works, it could have been more generic to explore the field better and compare i.e. more ctys close to finds multi. lots to combine to one big cluster of cheap neighbourhoods .... In extention to find i.e. 2 lots for combination could have been added for Seattle ( to not leave that marked untouched ..) 

is the lot size realy needed and is the price limit not still to high. Things to explore... the price limit reduction may help with outlyer in some features.

The map plot is very basic and is not using any information from arrayers (like mean price zip code etc.) in the map. A map with a raster rather then all data points as scatter could be helpfull. The map shape file as shape rather then a outline may results in more insight ... 




